In [0]:
# setup
!wget https://swcarpentry.github.io/python-novice-inflammation/data/python-novice-inflammation-data.zip
!wget https://swcarpentry.github.io/python-novice-inflammation/code/python-novice-inflammation-code.zip

!unzip -jn python-novice-inflammation-data.zip
!unzip -jn python-novice-inflammation-code.zip

All materials are from [Software Carpentry Lesson 12](https://swcarpentry.github.io/python-novice-inflammation/12-cmdline/index.html)

# Command-Line Programs

The Jupyter Notebook and other interactive tools are great for prototyping code and exploring data, but sooner or later we will want to use our program in a pipeline or run it in a shell script to process thousands of data files. In order to do that, we need to make our programs work like other Unix command-line tools. For example, we may want a program that reads a dataset and prints the average inflammation per patient.

### Switching to Shell Commands

In this lesson we are switching from typing commands in a Python interpreter to typing commands in a shell terminal window (such as bash). When you see a $ in front of a command that tells you to run that command in the shell rather than the Python interpreter.

This program does exactly what we want - it prints the average inflammation per patient for a given file.

In [5]:
!python readings_04.py --mean inflammation-01.csv

5.45
5.425
6.1
5.9
5.55
6.225
5.975
6.65
6.625
6.525
6.775
5.8
6.225
5.75
5.225
6.3
6.55
5.7
5.85
6.55
5.775
5.825
6.175
6.1
5.8
6.425
6.05
6.025
6.175
6.55
6.175
6.35
6.725
6.125
7.075
5.725
5.925
6.15
6.075
5.75
5.975
5.725
6.3
5.9
6.75
5.925
7.225
6.15
5.95
6.275
5.7
6.1
6.825
5.975
6.725
5.7
6.25
6.4
7.05
5.9


We might also want to look at the minimum of the first four lines

In [8]:
! head -4 inflammation-01.csv | python readings_06.py --min

0.0
0.0
0.0
0.0


or the maximum inflammations in several files one after another:

In [7]:
!python readings_04.py --max inflammation-*.csv

18.0
18.0
19.0
17.0
17.0
18.0
17.0
20.0
17.0
18.0
18.0
18.0
17.0
16.0
17.0
18.0
19.0
19.0
17.0
19.0
19.0
16.0
17.0
15.0
17.0
17.0
18.0
17.0
20.0
17.0
16.0
19.0
15.0
15.0
19.0
17.0
16.0
17.0
19.0
16.0
18.0
19.0
16.0
19.0
18.0
16.0
19.0
15.0
16.0
18.0
14.0
20.0
17.0
15.0
17.0
16.0
17.0
19.0
18.0
18.0
15.0
15.0
18.0
18.0
19.0
18.0
17.0
14.0
14.0
18.0
15.0
18.0
19.0
19.0
17.0
17.0
15.0
17.0
20.0
14.0
14.0
14.0
19.0
16.0
17.0
17.0
14.0
14.0
15.0
19.0
17.0
18.0
19.0
15.0
19.0
15.0
17.0
16.0
20.0
16.0
17.0
18.0
18.0
19.0
17.0
17.0
18.0
18.0
17.0
19.0
15.0
18.0
14.0
17.0
19.0
16.0
17.0
19.0
20.0
18.0
15.0
17.0
14.0
13.0
15.0
15.0
16.0
17.0
16.0
19.0
14.0
14.0
16.0
14.0
16.0
15.0
14.0
18.0
16.0
16.0
20.0
20.0
16.0
15.0
16.0
15.0
19.0
16.0
17.0
18.0
15.0
16.0
20.0
16.0
17.0
15.0
20.0
16.0
18.0
18.0
17.0
18.0
16.0
15.0
19.0
15.0
16.0
15.0
16.0
19.0
14.0
19.0
13.0
15.0
18.0
19.0
14.0
16.0
17.0
0.0
17.0
17.0
17.0
17.0
18.0
18.0
19.0
20.0
16.0
19.0
19.0
20.0
16.0
16.0
17.0
20.0
19.0
17.0
19.0
18.0
1

Our scripts should do the following:

1. If no filename is given on the command line, read data from [standard input](https://swcarpentry.github.io/python-novice-inflammation/reference/#standard-input).
2. If one or more filenames are given, read data from them and report statistics for each file separately.
Use the `--min`, `--mean`, or `--max` flag to determine what statistic to print.
3. To make this work, we need to know how to handle command-line arguments in a program, and understand how to handle standard input. We’ll tackle these questions in turn below.

## Command-Line Arguments

Using the text editor of your choice, save the following in a text file called `sys_version.py`:

In [9]:
import sys
print('version is', sys.version)

version is 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


The first line imports a library called `sys`, which is short for “system”. It defines values such as `sys.version`, which describes which version of Python we are running. We can run this script from the command line like this:

In [10]:
!python sys_version.py

version is 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


Create another file called `argv_list.py` and save the following text to it.

In [11]:
import sys
print('sys.argv is', sys.argv)

sys.argv is ['/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-730a53a0-f2d1-4371-8bb3-0f45ab2f426b.json']


The strange name `argv` stands for “argument values”. Whenever Python runs a program, it takes all of the values given on the command line and puts them in the list `sys.argv` so that the program can determine what they were. If we run this program with no arguments:

In [12]:
!python argv_list.py

sys.argv is ['argv_list.py']


the only thing in the list is the full path to our script, which is always `sys.argv[0]`. If we run it with a few arguments, however:

In [13]:
!python argv_list.py first second third

sys.argv is ['argv_list.py', 'first', 'second', 'third']


then Python adds each of those arguments to that magic list.

With this in hand, let’s build a version of `readings.py` that always prints the per-patient mean of a single data file. The first step is to write a function that outlines our implementation, and a placeholder for the function that does the actual work. By convention this function is usually called `main`, though we can call it whatever we want:

In [14]:
!cat readings_01.py

import sys
import numpy


def main():
    script = sys.argv[0]
    filename = sys.argv[1]
    data = numpy.loadtxt(filename, delimiter=',')
    for m in data.mean(axis=1):
        print(m)


This function gets the name of the script from `sys.argv[0]`, because that’s where it’s always put, and the name of the file to process from `sys.argv[1]`. Here’s a simple test:

In [0]:
!python readings_01.py inflammation-01.csv

There is no output because we have defined a function, but haven’t actually called it. Let’s add a call to `main`:

In [17]:
!cat readings_02.py

import sys
import numpy


def main():
    script = sys.argv[0]
    filename = sys.argv[1]
    data = numpy.loadtxt(filename, delimiter=',')
    for m in data.mean(axis=1):
        print(m)


if __name__ == '__main__':
    main()


In [20]:
!python readings_02.py inflammation-01.csv

5.45
5.425
6.1
5.9
5.55
6.225
5.975
6.65
6.625
6.525
6.775
5.8
6.225
5.75
5.225
6.3
6.55
5.7
5.85
6.55
5.775
5.825
6.175
6.1
5.8
6.425
6.05
6.025
6.175
6.55
6.175
6.35
6.725
6.125
7.075
5.725
5.925
6.15
6.075
5.75
5.975
5.725
6.3
5.9
6.75
5.925
7.225
6.15
5.95
6.275
5.7
6.1
6.825
5.975
6.725
5.7
6.25
6.4
7.05
5.9


### Running Versus Importing

Running a Python script in bash is very similar to importing that file in Python. The biggest difference is that we don’t expect anything to happen when we import a file, whereas when running a script, we expect to see some output printed to the console.

In order for a Python script to work as expected when imported or when run as a script, we typically put the part of the script that produces output in the following if statement:

```
if __name__ == '__main__':
    main()  # Or whatever function produces output
```

When you import a Python file, `__name__` is the name of that file (e.g., when importing `readings.py`, `__name__` is 'readings'). However, when running a script in bash, `__name__` is always set to `'__main__'` in that script so that you can determine if the file is being imported or run as a script.

### The Right Way to Do It

If our programs can take complex parameters or multiple filenames, we shouldn’t handle `sys.argv` directly. Instead, we should use Python’s `argparse` library, which handles common cases in a systematic way, and also makes it easy for us to provide sensible error messages for our users. We will not cover this module in this lesson but you can go to [Tshepang Lekhonkhobe’s Argparse tutorial](https://docs.python.org/3/howto/argparse.html) that is part of Python’s Official Documentation.

## Handling Multiple Files

The next step is to teach our program how to handle multiple files. Since 60 lines of output per file is a lot to page through, we’ll start by using three smaller files, each of which has three days of data for two patients:

In [21]:
!ls small-*.csv

small-01.csv  small-02.csv  small-03.csv


In [22]:
cat small-01.csv

0,0,1
0,1,2


In [23]:
!python readings_02.py small-01.csv

0.3333333333333333
1.0


Using small data files as input also allows us to check our results more easily: here, for example, we can see that our program is calculating the mean correctly for each line, whereas we were really taking it on faith before. This is yet another rule of programming: *test the simple things first*.

We want our program to process each file separately, so we need a loop that executes once for each filename. If we specify the files on the command line, the filenames will be in sys.argv, but we need to be careful: `sys.argv[0]` will always be the name of our script, rather than the name of a file. We also need to handle an unknown number of filenames, since our program could be run for any number of files.

The solution to both problems is to loop over the contents of `sys.argv[1:]`. The ‘1’ tells Python to start the slice at location 1, so the program’s name isn’t included; since we’ve left off the upper bound, the slice runs to the end of the list, and includes all the filenames. Here’s our changed program `readings_03.py`:

In [24]:
!cat readings_03.py

import sys
import numpy


def main():
    script = sys.argv[0]
    for filename in sys.argv[1:]:
        data = numpy.loadtxt(filename, delimiter=',')
        for m in data.mean(axis=1):
            print(m)


if __name__ == '__main__':
    main()


and here it is in action:

In [25]:
!python readings_03.py small-01.csv small-02.csv

0.3333333333333333
1.0
13.666666666666666
11.0


### The Right Way to Do It

At this point, we have created three versions of our script called `readings_01.py`, `readings_02.py`, and `readings_03.py`. We wouldn’t do this in real life: instead, we would have one file called `readings.py` that we committed to version control every time we got an enhancement working. For teaching, though, we need all the successive versions side by side.

## Handling Command-Line Flags

The next step is to teach our program to pay attention to the `--min`, `--mean`, and `--max` flags. These always appear before the names of the files, so we could do this:



In [26]:
!cat readings_04.py

import sys
import numpy


def main():
    script = sys.argv[0]
    action = sys.argv[1]
    filenames = sys.argv[2:]

    for f in filenames:
        data = numpy.loadtxt(f, delimiter=',')

        if action == '--min':
            values = data.min(axis=1)
        elif action == '--mean':
            values = data.mean(axis=1)
        elif action == '--max':
            values = data.max(axis=1)

        for m in values:
            print(m)


if __name__ == '__main__':
    main()


This works:

In [27]:
!python readings_04.py --max small-01.csv

1.0
2.0


but there are several things wrong with it:

1. `main` is too large to read comfortably.

2. If we do not specify at least two additional arguments on the command-line, one for the **flag** and one for the **filename**, but only one, the program will not throw an exception but will run. It assumes that the file list is empty, as `sys.argv[1]` will be considered the `action`, even if it is a filename. [Silent failures](https://swcarpentry.github.io/python-novice-inflammation/reference/#silence-failure) like this are always hard to debug.

3. The program should check if the submitted `action` is one of the three recognized flags.

This version pulls the processing of each file out of the loop into a function of its own. It also checks that `action` is one of the allowed flags before doing any processing, so that the program fails fast:

In [28]:
!cat readings_05.py

import sys
import numpy


def main():
    script = sys.argv[0]
    action = sys.argv[1]
    filenames = sys.argv[2:]
    assert action in ['--min', '--mean', '--max'], (
        'Action is not one of --min, --mean, or --max: ' + action)
    for f in filenames:
        process(f, action)


def process(filename, action):
    data = numpy.loadtxt(filename, delimiter=',')

    if action == '--min':
        values = data.min(axis=1)
    elif action == '--mean':
        values = data.mean(axis=1)
    elif action == '--max':
        values = data.max(axis=1)

    for m in values:
        print(m)


if __name__ == '__main__':
    main()


This is four lines longer than its predecessor, but broken into more digestible chunks of 8 and 12 lines.

## Handling Standard Input

The next thing our program has to do is read data from standard input if no filenames are given so that we can put it in a pipeline, redirect input to it, and so on. Let’s experiment in another script called `count_stdin.py`:

In [29]:
!cat count_stdin.py

import sys

count = 0
for line in sys.stdin:
    count += 1

print(count, 'lines in standard input')


This little program reads lines from a special “file” called `sys.stdin`, which is automatically connected to the program’s standard input. We don’t have to open it — Python and the operating system take care of that when the program starts up — but we can do almost anything with it that we could do to a regular file. Let’s try running it as if it were a regular command-line program:

In [30]:
!python count_stdin.py < small-01.csv

2 lines in standard input


A common mistake is to try to run something that reads from standard input like this:

In [31]:
!python count_stdin.py small-01.csv

Traceback (most recent call last):
  File "count_stdin.py", line 4, in <module>
    for line in sys.stdin:
KeyboardInterrupt
^C


i.e., to forget the `<` character that redirects the file to standard input. In this case, there’s nothing in standard input, so the program waits at the start of the loop for someone to type something on the keyboard. Since there’s no way for us to do this, our program is stuck, and we have to halt it using the `Interrupt` option from the `Kernel` menu in the Notebook.

We now need to rewrite the program so that it loads data from `sys.stdin` if no filenames are provided. Luckily, `numpy.loadtxt` can handle either a filename or an open file as its first parameter, so we don’t actually need to change `process`. Only `main` changes:

In [32]:
!cat readings_06.py

import sys
import numpy


def main():
    script = sys.argv[0]
    action = sys.argv[1]
    filenames = sys.argv[2:]
    assert action in ['--min', '--mean', '--max'], (
        'Action is not one of --min, --mean, or --max: ' + action)
    if len(filenames) == 0:
        process(sys.stdin, action)
    else:
        for f in filenames:
            process(f, action)


def process(filename, action):
    data = numpy.loadtxt(filename, delimiter=',')

    if action == '--min':
        values = data.min(axis=1)
    elif action == '--mean':
        values = data.mean(axis=1)
    elif action == '--max':
        values = data.max(axis=1)

    for m in values:
        print(m)


if __name__ == '__main__':
    main()


Let’s try it out:

In [34]:
!python readings_06.py --mean < small-01.csv

0.3333333333333333
1.0


That’s better. In fact, that’s done: the program now does everything we set out to do.

## Exercise: Arithmetic on the Command Line
Write a command-line program that does addition and subtraction:

```
$ python arith.py add 1 2
```
Output
```
3
```

```
$ python arith.py subtract 3 4
```
Output
```
-1
```

## Exercise: Finding Particular Files
Using the glob module introduced earlier, write a simple version of ls that shows files in the current directory with a particular suffix. A call to this script should look like this:

```
$ python my_ls.py py
```
Output
```
left.py
right.py
zero.py
```

## Exercise: Changing Flags
Rewrite `readings.py` so that it uses `-n`, `-m`, and `-x` instead of `--min`, `--mean`, and `--max` respectively. Is the code easier to read? Is the program easier to understand?

## Exercise: Adding a Help Message
Separately, modify `readings.py` so that if no parameters are given (i.e., no action is specified and no filenames are given), it prints a message explaining how it should be used.

## Exercise: Adding a Default Action
Separately, modify `readings.py` so that if no action is given it displays the means of the data.

## Exercise: A File-Checker
Write a program called `check.py` that takes the names of one or more inflammation data files as arguments and checks that all the files have the same number of rows and columns. What is the best way to test your program?

## Exercise: Counting Lines
Write a program called `line_count.py` that works like the Unix wc command:

* If no filenames are given, it reports the number of lines in standard input.
* If one or more filenames are given, it reports the number of lines in each, followed by the total number of lines.

## Exercise: Generate an Error Message
Write a program called `check_arguments.py` that prints usage then exits the program if no arguments are provided. (Hint: You can use `sys.exit()` to exit the program.)
```
$ python check_arguments.py
```

Output
```
usage: python check_argument.py filename.txt
```

```
$ python check_arguments.py filename.txt
```

Output
```
Thanks for specifying arguments!
```


## Exercise: Key Points

* The `sys` library connects a Python program to the system it is running on.

* The list `sys.argv` contains the command-line arguments that a program was run with.

* Avoid silent failures.

* The pseudo-file `sys.stdin` connects to a program’s standard input.